In [1]:
import numpy as np
import tensorflow as tf

In [2]:
n_features = 4
m_examples = 50

We have $X$, $m$ examples. Each example (each row) has $n$ features. Thus we have:
$$ 
\left[\begin{array}{cccc}
x^{(1)}_{1} & x^{(1)}_{2} & \ldots & x^{(1)}_{n} \\
x^{(2)}_{1} & x^{(2)}_{2} & \ldots & x^{(2)}_{n} \\
& & \vdots \\
x^{(m)}_{1} & x^{(m)}_{2} & \ldots & x^{(m)}_{n}
\end{array}\right] \times
\left[\begin{array}{c}
w_1 \\ w_2 \\ \vdots \\ w_n
\end{array}\right] + b =
\left[\begin{array}{c}
y_1 \\ y_2 \\ \vdots \\ v_n
\end{array}\right]
$$

In [3]:
# This is what we wish linear regression to learn.
w_true = np.random.rand(n_features, 1) * 100.0
# The constant
b_true = np.random.rand(1) * 100.0 - 50.0

print "w = [%s]" % ', '.join(['%.2f' % x for x in w_true])
print "b = %.2f" % b_true[0]

w = [63.67, 90.17, 76.85, 66.68]
b = -33.51


In [4]:
def create_training_fn(m, w, b):
  """Creates a training function for the given parameters.
  
  Args:
    m: int
      the number of examples to generate.
    w: np.array
      the weights used to compute y vector.
    b: float
      the bias added to X * w
  Return:
    function() -> (dict, tensor) training function.
  """
  def training_fn_():
    X = np.random.rand(m, w.shape[0])
    return ({'X': tf.constant(X)},
            tf.constant(np.matmul(X, w) + b))
  return training_fn_

In [5]:
# In version 1.0.x tf.contrib.layers are very verbose. To quiet
# them down only log errors.
tf.logging.set_verbosity(tf.logging.ERROR)
# Rather than having 4 features, we create one feature,
# 4 columns wide.
x_feature = tf.contrib.layers.real_valued_column('X', 4)
regressor = tf.contrib.learn.LinearRegressor(
  [x_feature], 
  optimizer=tf.train.GradientDescentOptimizer(0.3))
regressor.fit(
  input_fn=create_training_fn(m_examples, w_true, b_true),
  steps=500)
eval_dict = regressor.evaluate(
  input_fn=create_training_fn(10, w_true, b_true), steps=1)

In [6]:
# Used to have uniformly formatted printout.
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

print "loss", eval_dict['loss']
print "w true ", w_true.T[0]
print "w found", regressor.get_variable_value('linear/X/weight').T[0]
print "b true  %.4f" % b_true[0]
print "b found", regressor.get_variable_value('linear/bias_weight')[0]

loss 1.50816e-08
w true  [ 63.6656  90.1664  76.8489  66.6833]
w found [ 63.6653  90.1663  76.8487  66.6829]
b true  -33.5137
b found -33.5132
